In [2]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from functools import reduce
import pickle

# Dataset statistics

In [3]:
train_EN = pd.read_csv('../../data/multiATIS/raw/train_dev_test/train_EN.tsv',sep='\t')
#train_FR = pd.read_csv('../data/raw/train_dev_test/train_FR.tsv',sep='\t')
#train_DE = pd.read_csv('../data/raw/train_dev_test/train_DE.tsv',sep='\t')
#train_ES = pd.read_csv('../data/raw/train_dev_test/train_ES.tsv',sep='\t')

In [3]:
dev_EN = pd.read_csv('../data/raw/train_dev_test/dev_EN.tsv',sep='\t')
dev_FR = pd.read_csv('../data/raw/train_dev_test/dev_FR.tsv',sep='\t')
dev_DE = pd.read_csv('../data/raw/train_dev_test/dev_DE.tsv',sep='\t')
dev_ES = pd.read_csv('../data/raw/train_dev_test/dev_ES.tsv',sep='\t')

In [5]:
# freq dist of classes in trainset and devset
trainIntent = Counter(train_EN.intent)
#devIntent = Counter(dev_EN.intent)

In [7]:
print('Classes in trainset:',trainIntent)
#print('\n\nClasses in devset:',devIntent)

Classes in trainset: Counter({'atis_flight': 3300, 'atis_airfare': 381, 'atis_ground_service': 230, 'atis_airline': 142, 'atis_abbreviation': 133, 'atis_aircraft': 73, 'atis_flight_time': 49, 'atis_quantity': 46, 'atis_flight#atis_airfare': 19, 'atis_distance': 18, 'atis_airport': 18, 'atis_city': 18, 'atis_ground_fare': 17, 'atis_capacity': 15, 'atis_flight_no': 11, 'atis_restriction': 6, 'atis_meal': 6, 'atis_airline#atis_flight_no': 2, 'atis_ground_service#atis_ground_fare': 1, 'atis_airfare#atis_flight_time': 1, 'atis_cheapest': 1, 'atis_aircraft#atis_flight#atis_flight_no': 1})


In [11]:
#train_EN['TEXT'] = train_EN['TEXT'].str.lower()
#train_DE['utterance'] = train_DE['utterance'].str.lower()
#train_ES['utterance'] = train_ES['utterance'].str.lower()
train_EN['utterance'] = train_EN['utterance'].str.lower()

#dev_FR['utterance'] = dev_FR['utterance'].str.lower()
#dev_DE['utterance'] = dev_DE['utterance'].str.lower()
#dev_ES['utterance'] = dev_ES['utterance'].str.lower()
#dev_EN['utterance'] = dev_EN['utterance'].str.lower()

In [12]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no'}
dev_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare'}

In [13]:
# remapping classes 
train_EN['intent'] = train_EN['intent'].replace(train_replace_dict)
#train_FR['intent'] = train_FR['intent'].replace(train_replace_dict)
#train_ES['intent'] = train_ES['intent'].replace(train_replace_dict)
#train_DE['intent'] = train_DE['intent'].replace(train_replace_dict)

#dev_EN['intent'] = dev_EN['intent'].replace(dev_replace_dict)
#dev_FR['intent'] = dev_FR['intent'].replace(dev_replace_dict)
#dev_ES['intent'] = dev_ES['intent'].replace(dev_replace_dict)
#dev_DE['intent'] = dev_DE['intent'].replace(dev_replace_dict)

In [15]:
trainIntent = Counter(train_EN.intent)
#devIntent = Counter(dev_EN.intent)
print('Classes in trainset:',trainIntent.keys())
#print('\n\nClasses in devset:',devIntent)

Classes in trainset: dict_keys(['atis_airfare', 'atis_flight_no', 'atis_flight', 'atis_capacity', 'atis_quantity', 'atis_distance', 'atis_restriction', 'atis_aircraft', 'atis_meal', 'atis_ground_fare', 'atis_airport', 'atis_flight_time', 'atis_ground_service', 'atis_cheapest', 'atis_abbreviation', 'atis_airline', 'atis_city'])


In [11]:
# generating global intent mapping to idx.

intent_list = list(train_EN.intent.unique()) #list(set(list(train_EN.intent.unique())+ list(train_FR.intent.unique())))
map_intent_idx = { intent:idx for idx,intent in enumerate(intent_list)} 
# handling OOI
#map_intent_idx['OOI'] = len(map_intent_idx.keys()) 
print(map_intent_idx.keys(),len(map_intent_idx.keys()))

dict_keys(['atis_airfare', 'atis_flight_no', 'atis_flight', 'atis_capacity', 'atis_quantity', 'atis_distance', 'atis_restriction', 'atis_aircraft', 'atis_meal', 'atis_ground_fare', 'atis_airport', 'atis_flight_time', 'atis_ground_service', 'atis_cheapest', 'atis_abbreviation', 'atis_airline', 'atis_city']) 17


In [12]:
#generating global mapping from slot to idx.
slots = []
for annotations in list(train_EN.slot_labels.unique()):
    ann = annotations.replace('B-','')
    ann = ann.replace('I-','')
    slots += ann.split()

slots = list(set(slots))
final_slots = []
for s in slots:
    if s != 'O':
        #print('B-'+s , 'I-'+s)
        final_slots += ['B-'+s , 'I-'+s]
    else:
        final_slots.append('O')

map_slots_idx  = {slots:idx for idx,slots in enumerate(final_slots)}
print(len(map_slots_idx.keys()))

159


In [68]:
map_idx_slots = {v: k for k, v in map_slots_idx.items()}

In [70]:
with open('map_ids_slots.pickle', 'wb') as handle:
    pickle.dump(map_idx_slots, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
map_slots_idx['O']

82

In [40]:
def processing_mapping(df,map_intent_idx,map_slots_idx,fn):
    # data processing and encoding.
    #lowercasing utterance column text
    #df = df.utterance.str.lower()
    del df['id']

    # adding intent encoding
    encoded_intent = []
    for intent in list(df.intent):
        try:
            encoded_intent.append(map_intent_idx[intent])
        except:
            print(intent)
            encoded_intent.append(map_intent_idx['OOI'])
            
        
    df['intent_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.slot_labels):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(map_slots_idx[tokens]) + " "
            except:
                print('token',tokens,fn)
                encoded_slot += str(map_slots_idx['O'])
        slots_ID.append(" ".join(encoded_slot.split()))

    df['slots_ID'] = slots_ID
    
    return df

In [41]:
train_EN = processing_mapping(train_EN,map_intent_idx,map_slots_idx,'tr_en')
train_FR = processing_mapping(train_FR,map_intent_idx,map_slots_idx,'tr_fr')
train_DE = processing_mapping(train_DE,map_intent_idx,map_slots_idx,'tr_de')
train_ES = processing_mapping(train_ES,map_intent_idx,map_slots_idx,'tr_es')

dev_EN = processing_mapping(dev_EN,map_intent_idx,map_slots_idx,'dev_en')
dev_FR = processing_mapping(dev_FR,map_intent_idx,map_slots_idx,'dev_fr')
dev_ES = processing_mapping(dev_ES,map_intent_idx,map_slots_idx,'dev_es')
dev_DE = processing_mapping(dev_DE,map_intent_idx,map_slots_idx,'dev_de')

In [42]:
train_DE

,utterance,slot_labels,intent,intent_ID,slots_ID
0,was kostet der american airlines -flug 71 von ...,O O O B-airline_name I-airline_name O B-flight...,atis_airfare,0,82 82 82 149 150 82 2 82 38 82 155 156
1,kannst du mir den günstigsten tarif von atlant...,O O O O B-cost_relative O O B-fromloc.city_nam...,atis_airfare,0,82 82 82 82 87 82 82 38 82 155 82 95 103 82
2,was kostet der flug dl 106 von philadelphia na...,O O O O B-airline_code B-flight_number O B-fro...,atis_airfare,0,82 82 82 82 91 2 82 38 82 155
3,was ist der günstigste tarif für einen hin- un...,O O O B-cost_relative O O O B-round_trip I-rou...,atis_airfare,0,82 82 82 87 82 82 82 46 47 47 82 82 38 82 155 156
4,was ist der tarif von boston nach oakland mit ...,O O O O O B-fromloc.city_name O B-toloc.city_n...,atis_airfare,0,82 82 82 82 82 38 82 155 82 149 150
...,...,...,...,...,...
4483,bitte zeige mir die flugzeiten von newark nach...,O O O O B-flight_time O B-fromloc.city_name O ...,atis_flight_time,11,82 82 82 82 111 82 38 82 155
4484,ich möchte die uhrzeit des spätesten fluges wi...,O O O B-flight_time O B-flight_mod O O O O O B...,atis_flight_time,11,82 82 82 111 82 58 82 82 82 82 82 38 82 155 15...
4485,um wie viel uhr gibt es den letzten flug von w...,O O O B-flight_time O O O B-flight_mod O O B-f...,atis_flight_time,11,82 82 82 111 82 82 82 58 82 82 38 82 155 156
4486,"in ordnung, zeige mir die flugzeiten morgens a...",O O O O O B-flight_time B-depart_time.period_o...,atis_flight_time,11,82 82 82 82 82 111 52 82 95 103 82 38 82 155 156


In [59]:
# saving merged dataset for multi-lingual nlu modelling.
train_dataset_1 = pd.concat([train_EN,train_DE,train_ES,train_FR])
train_dataset_1['language'] = ['en']*len(train_EN)+['de']*len(train_DE)+['es']*len(train_ES)+['fr']*len(train_FR)

dev_dataset_1 = pd.concat([dev_EN,dev_DE,dev_ES,dev_FR])
dev_dataset_1['language'] = ['en']*len(dev_EN)+['de']*len(dev_DE)+['es']*len(dev_ES)+['fr']*len(dev_FR)

train_dataset_1['ID'] = range(len(train_dataset_1))
train_dataset_1.set_index('ID',inplace=True)

dev_dataset_1['ID'] = range(len(dev_dataset_1))
dev_dataset_1.set_index('ID',inplace=True)
# writing dataframe to a file
train_dataset_1.to_csv('../data/splits/multi-train.tsv',sep='\t',index=False)
dev_dataset_1.to_csv('../data/splits/multi-dev.tsv',sep='\t',index=False)

In [25]:
# saving isolated dataset
train_EN.to_csv('../data/splits/train_EN.tsv',sep='\t',index=False)
train_ES.to_csv('../data/splits/train_ES.tsv',sep='\t',index=False)
train_DE.to_csv('../data/splits/train_DE.tsv',sep='\t',index=False)
train_FR.to_csv('../data/splits/train_FR.tsv',sep='\t',index=False)

dev_EN.to_csv('../data/splits/dev_EN.tsv',sep='\t',index=False)
dev_ES.to_csv('../data/splits/dev_ES.tsv',sep='\t',index=False)
dev_DE.to_csv('../data/splits/dev_DE.tsv',sep='\t',index=False)
dev_FR.to_csv('../data/splits/dev_FR.tsv',sep='\t',index=False)

In [75]:
multi_train = pd.read_csv('../data/splits/multi-train.tsv',sep='\t')
multi_train = multi_train.sample(frac=1)
multi_train

multi_dev = pd.read_csv('../data/splits/multi-dev.tsv',sep='\t')
multi_dev = multi_dev.sample(frac=1)
multi_dev

multi_train.to_csv('../data/splits/multi-train.tsv',sep='\t',index=False)
multi_dev.to_csv('../data/splits/multi-dev.tsv',sep='\t',index=False)
print(len(multi_train),len(multi_dev))

17952 1960


# Generating testset for tsne plot

In [ ]:
testEN = pd.read_csv('../data/raw/train_dev_test/test_EN.tsv', sep='\t')
testEN = testEN.rename(columns={"id": "id_EN", "utterance": "utterance_EN" , "slot_labels":"slot_labels_EN" , "intent":"intent_EN"})
testFR = pd.read_csv('../data/raw/train_dev_test/test_FR.tsv', sep='\t')
testFR = testFR.rename(columns={"id": "id_FR", "utterance": "utterance_FR" , "slot_labels":"slot_labels_FR" , "intent":"intent_FR"})

In [ ]:
testEN

In [ ]:
testFR

In [ ]:
testData = pd.concat([testEN,testFR],axis=1)
testData = testData[['id_EN','utterance_EN','slot_labels_EN','intent_EN','utterance_FR','slot_labels_FR']]

In [ ]:
testData

In [ ]:
testData['intent_EN'] = testData['intent_EN'].replace({'atis_flight#atis_airfare': 'atis_airfare',
                                              'atis_airfare#atis_flight' : 'atis_airfare',
                                              'atis_flight_no#atis_airline' : 'atis_flight_no',
                                               'atis_flight#atis_airline' : 'atis_airline'})

In [ ]:
testData

In [ ]:
data_dict = { intent : [] for intent in testData.intent_EN.unique()}
testData = testData.sample(frac=1)
for i in range(len(testData)):
    if len(data_dict[testData.intent_EN.iloc[i]])>=10:
        continue
    else:
        data_dict[testData.intent_EN.iloc[i]].append([testData.utterance_EN.iloc[i],testData.utterance_FR.iloc[i]])

In [ ]:
for keys,value in data_dict.items():
    print(keys,len(value))

In [ ]:
data_dict.pop('atis_quantity')
data_dict.pop('atis_flight_time')
data_dict.pop('atis_day_name')

In [ ]:
Intent,Utt,Lang = [],[],[]
for intent,utt in data_dict.items():
    for sent in utt:
        Intent += [intent]*2
        Utt+= sent
        Lang += ['en','fr']

In [ ]:
len(Intent), len(Lang) , len(Utt)

In [ ]:
vis_data = pd.DataFrame({'LANGUAGE':Lang,'UTTERANCE':Utt,'INTENT':Intent})
vis_data

In [ ]:
vis_data.to_csv('../data/splits/text/TSNE_data.tsv',sep='\t',index=False)

# distribution plot

In [ ]:
trainEN = pd.read_csv('./data/splits/train_EN.tsv',sep='\t')
trainEN.head()

In [ ]:
grouped_data

In [ ]:
TYPE = ['Train']*17 + ['Dev'] *14 + ['Test']*16

In [ ]:
Intent = list(train_CTR.keys())+list(dev_CTR.keys())+list(test_CTR.keys())

In [ ]:
Freq = list(train_CTR.values()) + list(dev_CTR.values()) + list(test_CTR.values())

In [ ]:
dataPD = pd.DataFrame({'TYPE':TYPE , 'Intent': Intent , 'Freq':Freq})

In [ ]:
dataPD['Freq'] = np.log2(dataPD['Freq']) + 1 

In [ ]:
plot = sns.catplot(x='Intent', y='Freq', hue='TYPE', data=dataPD, kind='bar',height=6.27, aspect=11.7/8.27, alpha=1.0)
plot.set_xticklabels(rotation=90)

In [ ]:
data_EN.to_csv('./data/splits/train_EN.tsv' , sep='\t',index=None)
dev_EN.to_csv('./data/splits/dev_EN.tsv' , sep='\t',index=None)
test_EN.to_csv('./data/splits/test_EN.tsv' , sep='\t',index=None)

# Dataset for back translation

In [ ]:
import pandas as pd

In [ ]:
trainEN = pd.read_csv('../data/splits/train_EN.tsv',sep='\t')
trainFR = pd.read_csv('../data/splits/train_FR.tsv',sep='\t')

In [ ]:
trainEN = trainEN[['utterance','Language']]
trainFR = trainFR[['utterance','Language']]

In [ ]:
train = pd.concat([trainEN,trainFR])
train.head